# Modules and Data


In [48]:
import csv
import datetime
import pandas as pd

# gun death data 
data = list(csv.reader(open("guns.csv", "r")))
headers = data[0]
data = data[1:]

# population census data 

census = list(csv.reader(open("census.csv", "r")))
census_headers = census[0] 
census = census[1:][0]



# Functions for Simple Calculations Used Across this Project 

In [8]:
def header_index(header, headers = headers):
    header_values = {}
    for index, value in enumerate(headers):
        header_values[value] = index
        
    header_index = header_values[header]
    return header_index

def get_column_counts(column, list = False):
    col = [col[header_index(column)] for col in data]
    if list == True: 
        return col
    col_count = {}
    for value in col:
        if value in col_count:
            col_count[value] += 1
        else:
            col_count[value] = 1
    return col_count

# Calculations from Data 

In [4]:
# calculate gun deaths by year 

year_counts = get_column_counts('year')
year_counts


{'2012': 33563, '2013': 33636, '2014': 33599}

In [139]:
# calcluate gun deaths by month

dates = [datetime.datetime(year = int(row[header_index('year')]), month = int(row[header_index('month')]), day = 1).strftime("%Y-%m") for row in data]
date_counts = {}
for date in dates: 
    if date in date_counts:
        date_counts[date] += 1
    else:
        date_counts[date] = 1
        



In [138]:
# look at sex and race columns
# this code is pretty bad.. haha. Was looking for it to be easy to read but you know... 

sex_counts = get_column_counts("sex")
race_counts = get_column_counts("race")
mapping = {}
race_per_hundredk = {}
for key, value in race_counts.items():
    if key == 'Asian/Pacific Islander':
        mapping[key] = (int(census[header_index('Race Alone - Asian', headers = census_headers)]) +
                       int(census[header_index('Race Alone - Native Hawaiian and Other Pacific Islander', headers = census_headers)]))
        race_per_hundredk[key] = 100000*(value/ mapping[key])
    elif key == 'Black':
        mapping[key] = int(census[header_index('Race Alone - Black or African American', headers = census_headers)])
        race_per_hundredk[key] = 100000*(value/ mapping[key])
    elif key == 'Hispanic':
        mapping[key] = int(census[header_index('Race Alone - Hispanic', headers = census_headers)])
        race_per_hundredk[key] = 100000*(value/ mapping[key])
    elif key == 'Native American/Native Alaskan':
        mapping[key] = int(census[header_index('Race Alone - American Indian and Alaska Native', headers = census_headers)])
        race_per_hundredk[key] = 100000*(value/ mapping[key])
    elif key == 'White': 
        mapping[key] = int(census[header_index('Race Alone - White', headers = census_headers)])
        race_per_hundredk[key] = 100000*(value/ mapping[key])
    

In [137]:
# Looking at gun deaths across the entire dataset by intent, race, per 100k 

def intent_per_race(intent, hundredk = False): 
    intent_per_race = {}
    intents = get_column_counts('intent', list = True)
    races = get_column_counts('race', list = True)
    for i, race in enumerate(races):
        if intents[i] == intent:
            if race in intent_per_race:
                intent_per_race[race] += 1
            else:
                intent_per_race[race] = 1
    if hundredk == True: 
        intent_per_race_hundredk = {}
        for key, value in intent_per_race.items():
            intent_per_race_hundredk[key] = 100000*(value/ int(mapping[key]))
        return intent_per_race_hundredk
    else:
        return intent_per_race

def intent_per(hundredk = False):             
    unique_intent = list(set([get_column_counts('intent', list = True)][0]))

    intent_per_hundredk = {}
    for intent in unique_intent: 
        if intent not in intent_per_hundredk:
            intent_per_hundredk[intent] = intent_per_race(intent, hundredk=hundredk)
        
    return intent_per_hundredk

# calcluated dictionaries 

intent_by_death_total = intent_per(hundredk = False)
intent_by_death_rate = intent_per(hundredk = True) 


    

In [136]:
#intent by month totals and per 100k

def intent_per_month(intent, hundredk = False):
    intents = get_column_counts('intent', list = True)
    races = get_column_counts('race', list = True)
    intent_per_month = {} 
    for date in list(set(dates)):
        if date not in intent_per_month:
            month_by_race = {}
            for j, race in enumerate(races):
                if intents[j] == intent:
                    if dates[j] == date:
                        if race in month_by_race:
                            month_by_race[race] += 1
                        else:
                            month_by_race[race] = 1
            
            if hundredk == True:
                for key, value in month_by_race.items(): 
                    month_by_race[key] = 100000*(value/int(mapping[key]))
            intent_per_month[date] = month_by_race
            
    return intent_per_month

def month_total_or_rate(hundredk = False):
    unique_intent = list(set([get_column_counts('intent', list = True)][0]))
    gun_death_per_month = {}
    for intent in unique_intent:
        if intent not in gun_death_per_month: 
            gun_death_per_month[intent] = intent_per_month(intent, hundredk=hundredk)
    return gun_death_per_month

#dictionary calculations 

intent_total_by_month = month_total_or_rate()
intent_total_by_100k = month_total_or_rate(hundredk = True)

